Voici les librairies utile pour le bon fonctionnement du code :

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import string
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix

from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score

from sklearn.utils import shuffle

from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.naive_bayes import GaussianNB

Voici les fonctions utilisés pour le preprocessing, on enlève les stopwords, on fait un stemmatization et on tokenize

In [3]:
stemming = PorterStemmer()
nltk.download('stopwords')
stops = set(stopwords.words("english"))
def Cremove_stops(row):
    my_list = row['comment']
    meaningful_words = [w for w in my_list if not w in stops]
    return (meaningful_words)

def Cstem_list(row):
    my_list = row['comment']
    stemmed_list = [stemming.stem(word) for word in my_list]
    return (stemmed_list)

def Cidentify_tokens(row):
    review = row['comment']
    tokens = nltk.word_tokenize(review)
    # taken only words (not punctuation)
    token_words = [w for w in tokens if w.isalpha()]
    return token_words

def Premove_stops(row):
    my_list = row['parent_comment']
    meaningful_words = [w for w in my_list if not w in stops]
    return (meaningful_words)

def Pstem_list(row):
    my_list = row['parent_comment']
    stemmed_list = [stemming.stem(word) for word in my_list]
    return (stemmed_list)

def Pidentify_tokens(row):
    review = row['parent_comment']
    tokens = nltk.word_tokenize(review)
    # taken only words (not punctuation)
    token_words = [w for w in tokens if w.isalpha()]
    return token_words

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


On va effectuer deux preprocessing différents, c'est pour quoi on prend de dataframes.

In [4]:
df = pd.read_csv("/content/drive/My Drive/NLPW/Train.csv", usecols=[1,3,4,5,6,7,9],header=0)
df2 = pd.read_csv("/content/drive/My Drive/NLPW/Train.csv", usecols=[1,3,4,5,6,7,9],header=0)
dfnb = pd.read_csv("/content/drive/My Drive/NLPW/Train.csv", usecols=[1,4,5,9],header=0)


On applique tout sur df, pour faire un premier preprocessing.

In [5]:
nltk.download('punkt')
df['comment'] = df.apply(Cidentify_tokens, axis=1)
df['comment'] = df.apply(Cremove_stops, axis=1)
df['comment'] = df.apply(Cstem_list, axis=1)

df['parent_comment'] = df.apply(Pidentify_tokens, axis=1)
df['parent_comment'] = df.apply(Premove_stops, axis=1)
df['parent_comment'] = df.apply(Pstem_list, axis=1)

dfnb['comment'] = dfnb.apply(Cidentify_tokens, axis=1)
dfnb['comment'] = dfnb.apply(Cremove_stops, axis=1)
dfnb['comment'] = dfnb.apply(Cstem_list, axis=1)

dfnb['parent_comment'] = dfnb.apply(Pidentify_tokens, axis=1)
dfnb['parent_comment'] = dfnb.apply(Premove_stops, axis=1)
dfnb['parent_comment'] = dfnb.apply(Pstem_list, axis=1)

print(dfnb)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
                                                 comment  ... label
0      [well, let, honest, actual, seem, much, moder,...  ...     0
1                  [well, I, need, evid, believ, commun]  ...     1
2                                   [who, offici, promo]  ...     0
3                                   [grotto, koth, best]  ...     1
4                                     [neal, back, babi]  ...     1
...                                                  ...  ...   ...
14995  [well, name, like, El, cubano, I, surpris, alr...  ...     0
14996                                 [thi, good, point]  ...     0
14997                                              [yep]  ...     0
14998                       [that, govern, want, believ]  ...     1
14999                     [window, gloriou, start, menu]  ...     1

[15000 rows x 4 columns]


Pour le second dataset, on va uniquement tokenizer.

In [6]:
df2['comment'] = df2.apply(Cidentify_tokens, axis=1)
df2['parent_comment'] = df2.apply(Pidentify_tokens, axis=1)

In [7]:
X = df.iloc[:, 0:6]
y = df["label"]

A = df2.iloc[:, 0:6]
b = df2["label"]
print(X)
print(y)

                                                 comment  ...             topic
0      [well, let, honest, actual, seem, much, moder,...  ...       starcitizen
1                  [well, I, need, evid, believ, commun]  ...  EnoughCommieSpam
2                                   [who, offici, promo]  ...       motorcycles
3                                   [grotto, koth, best]  ...        hcfactions
4                                     [neal, back, babi]  ...            hockey
...                                                  ...  ...               ...
14995  [well, name, like, El, cubano, I, surpris, alr...  ...            hockey
14996                                 [thi, good, point]  ...            hockey
14997                                              [yep]  ...         AskReddit
14998                       [that, govern, want, believ]  ...         AskReddit
14999                     [window, gloriou, start, menu]  ...         Windows10

[15000 rows x 6 columns]
0        0
1  

Appliquons notre modèle (Random Forest Classifier) uniquement sur df

In [8]:
kf = KFold(n_splits=3)
X = shuffle(X)

for train_index, test_index in kf.split(X):
    print("TRAIN:", train_index, "TEST:", test_index)

    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    X_train, y_train = make_classification(n_samples=10000, n_features=2,n_informative=2, n_redundant=0,random_state=2,shuffle=True)
    X_test, y_test = make_classification(n_samples=10000, n_features=2,n_informative=2, n_redundant=0,random_state=2,shuffle=True)
    clf = RandomForestClassifier(max_depth=3, random_state=0)
    clf.fit(X_train,y_train)
    y_pre= clf.predict(X_test)
    print("le f1score est : ",f1_score(y_test, y_pre,  average='binary'))
    print("la précision est : ", precision_score(y_test, y_pre,  average='binary'))
    print("le recall est : ",recall_score(y_test, y_pre,  average='binary'))
    print("le confusion matrix est :\n ",confusion_matrix(y_test, y_pre))
    


TRAIN: [ 5000  5001  5002 ... 14997 14998 14999] TEST: [   0    1    2 ... 4997 4998 4999]
le f1score est :  0.9451188923447376
la précision est :  0.9480676328502415
le recall est :  0.9421884376875375
le confusion matrix est :
  [[4743  258]
 [ 289 4710]]
TRAIN: [    0     1     2 ... 14997 14998 14999] TEST: [5000 5001 5002 ... 9997 9998 9999]
le f1score est :  0.9451188923447376
la précision est :  0.9480676328502415
le recall est :  0.9421884376875375
le confusion matrix est :
  [[4743  258]
 [ 289 4710]]
TRAIN: [   0    1    2 ... 9997 9998 9999] TEST: [10000 10001 10002 ... 14997 14998 14999]
le f1score est :  0.9451188923447376
la précision est :  0.9480676328502415
le recall est :  0.9421884376875375
le confusion matrix est :
  [[4743  258]
 [ 289 4710]]


Maintenant ,appliquons notre modèle uniquement sur df2

In [9]:
kf2 = KFold(n_splits=3)
A = shuffle(A)
for train_index, test_index in kf.split(A):
    print("TRAIN:", train_index, "TEST:", test_index)
    A_train, A_test = A.iloc[train_index], A.iloc[test_index]
    b_train, b_test = b.iloc[train_index], b.iloc[test_index]
    A_train, b_train = make_classification(n_samples=10000, n_features=2,n_informative=2, n_redundant=0,random_state=2,shuffle=True)
    A_test, b_test = make_classification(n_samples=10000, n_features=2,n_informative=2, n_redundant=0,random_state=2,shuffle=True)
    clf = RandomForestClassifier(max_depth=3, random_state=0)
    clf.fit(A_train,b_train)
    b_pre= clf.predict(A_test)
    print("le f1score est : ",f1_score(b_test, b_pre,  average='binary'))
    print("la précision est : ",precision_score(b_test, b_pre,  average='binary'))
    print("le recall est : ",recall_score(b_test, b_pre,  average='binary'))
    print("le confusion matrix est :\n ",confusion_matrix(y_test, y_pre))

TRAIN: [ 5000  5001  5002 ... 14997 14998 14999] TEST: [   0    1    2 ... 4997 4998 4999]
le f1score est :  0.9451188923447376
la précision est :  0.9480676328502415
le recall est :  0.9421884376875375
le confusion matrix est :
  [[4743  258]
 [ 289 4710]]
TRAIN: [    0     1     2 ... 14997 14998 14999] TEST: [5000 5001 5002 ... 9997 9998 9999]
le f1score est :  0.9451188923447376
la précision est :  0.9480676328502415
le recall est :  0.9421884376875375
le confusion matrix est :
  [[4743  258]
 [ 289 4710]]
TRAIN: [   0    1    2 ... 9997 9998 9999] TEST: [10000 10001 10002 ... 14997 14998 14999]
le f1score est :  0.9451188923447376
la précision est :  0.9480676328502415
le recall est :  0.9421884376875375
le confusion matrix est :
  [[4743  258]
 [ 289 4710]]


On remarque que le Stemmatizer et qu'enlever les stopwords ne change rien pour notre modèle

**///////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////*

**///////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////*

**///////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////*

**///////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////*

**///////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////*

**///////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////*

**///////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////*

Application de Naive Bayes:

We will initially use only 4 3 parameters: Comment, Parent comment and comment score since the rest can be hardly countabilised, that is to say cannot serve for Naive Bayes approach.

In [10]:
dfnb = pd.read_csv("/content/drive/My Drive/NLPW/Train.csv", usecols=[1,4,5,9],header=0)

dfnb['comment'] = dfnb.apply(Cidentify_tokens, axis=1)
dfnb['comment'] = dfnb.apply(Cremove_stops, axis=1)
dfnb['comment'] = dfnb.apply(Cstem_list, axis=1)

dfnb['parent_comment'] = dfnb.apply(Pidentify_tokens, axis=1)
dfnb['parent_comment'] = dfnb.apply(Premove_stops, axis=1)
dfnb['parent_comment'] = dfnb.apply(Pstem_list, axis=1)


Since Naive Bayes works with word counts, we will need to create a vocabulary for comment and parent_comment columns:

In [ ]:
vocabulary = []
for message in dfnb['comment']:
   for word in message:
      vocabulary.append(word)
for message in dfnb['parent_comment']:
  for word in message:
    vocabulary.append(word)
vocabulary = list(set(vocabulary))
print(vocabulary)
print('\n Unique words in vocabulary: '+str(len(vocabulary)))

['wavi', 'barbi', 'greeaat', 'combat', 'blog', 'thunderbolt', 'tackl', 'yeezi', 'biomarket', 'stif', 'kurt', 'situat', 'truer', 'quantit', 'mammal', 'itsi', 'trivia', 'compact', 'riot', 'aph', 'petrol', 'doppler', 'unplay', 'volt', 'uninterrupt', 'rwd', 'sausag', 'drummer', 'natuurlijk', 'homestuck', 'clover', 'settlement', 'u', 'field', 'ligament', 'acid', 'coma', 'firebird', 'indifer', 'gorno', 'PD', 'jung', 'glove', 'fantasi', 'DT', 'schol', 'bunch', 'rariti', 'oatmeal', 'goofbal', 'handcuff', 'syrian', 'shodan', 'duke', 'fanservic', 'latent', 'bryz', 'swtich', 'heck', 'pendulum', 'absterget', 'citat', 'champlo', 'strength', 'pack', 'nomad', 'ndp', 'pg', 'gooser', 'giancarlo', 'cater', 'preas', 'boldin', 'kinet', 'pakistan', 'wile', 'nebraska', 'cookiezi', 'buck', 'board', 'nitro', 'could', 'buyer', 'jarvan', 'cooki', 'bronu', 'iv', 'netapp', 'homegrown', 'disapoint', 'crore', 'phuket', 'muramana', 'duur', 'dock', 'solitud', 'coast', 'flee', 'tottenham', 'adept', 'heartach', 'quirk'

On va transformer notre dataframe grace au vocabulaire cree:

In [ ]:
df_cm=dfnb.drop(['parent_comment', 'label'], axis=1)
df_pcm=dfnb.drop(['comment', 'label'], axis=1)

word_counts_per_comment = {unique_word: [0] * len(df_cm['comment']) for unique_word in vocabulary}
word_counts_per_parent_comment = {unique_word: [0] * len(df_pcm['parent_comment']) for unique_word in vocabulary}

for index, comment in enumerate(df_cm['comment']):
   for word in comment:
      word_counts_per_comment[word][index] += 1

for index, comment in enumerate(df_pcm['parent_comment']):
   for word in comment:
      word_counts_per_parent_comment[word][index] += 1


In [ ]:
wc_df_c = pd.DataFrame(word_counts_per_comment)
#wc_df_p = pd.DataFrame(word_counts_per_parent_comment)
#wc_df_p.head()

In [ ]:
wc_df_c.to_csv(path_or_buf="/content/drive/My Drive/NLPW/wc_df_c.csv",sep=',')

La matrice s-parse etant beaucoup trop volumineuse pour pouvoir efficacement utiliser Naive-Bayes, on va utiliser un autre methode, en utilisant CNN